# BIOMASS TDS

## Functions and classes

In [1]:
import json
from dataclasses import dataclass
from typing import Optional

import numpy as np
import pandas as pd
from biotm import const, tm
from procsim.core.version import __version__ as procsim_version


# Definitions
ORBITS_PER_RC = 44  # 44 orbits in 3 days
ORBIT_DURATION = 86400 * 3 / 44
ORBITS_PER_SRM = int(ORBITS_PER_RC / 3 * 9)
SWATHS = 3
TOM_RC_PER_SWATH = 7
INT_RC_PER_SWATH = 3
TOM_ORBITS_PER_SWATH = ORBITS_PER_RC * TOM_RC_PER_SWATH
INT_ORBITS_PER_SWATH = ORBITS_PER_RC * INT_RC_PER_SWATH
TOM_ORBITS_PER_MC = TOM_ORBITS_PER_SWATH * SWATHS + ORBITS_PER_SRM
INT_ORBITS_PER_MC = INT_ORBITS_PER_SWATH * SWATHS + ORBITS_PER_SRM
TOM_ORBITS_PER_GC = TOM_ORBITS_PER_MC * 6 + TOM_ORBITS_PER_SWATH + ORBITS_PER_SRM
INT_ORBITS_PER_GC = INT_ORBITS_PER_MC * 6 + INT_ORBITS_PER_SWATH + ORBITS_PER_SRM
SAR_ISPS_PER_SEC = int(np.ceil(1 / 340e-6))  # Assumption: SAR PRI = 340 us
ANC_ISPS_PER_SEC = 1
BASELINE = 1
SLICE_MIN_DURATION = 28
FRAME_MIN_DURATION = 7


@dataclass
class AbsoluteOrbit:
    
    orbit: int
    mission_phase: tm.MissionPhase
    start: Optional[pd.Timestamp] = None
    stop: Optional[pd.Timestamp] = None
    
    def orbits_per_mc(self) -> int:
        if self.mission_phase == tm.MissionPhase.TOM:
            result = TOM_ORBITS_PER_MC
        elif self.mission_phase == tm.MissionPhase.INT:
            result = INT_ORBITS_PER_MC
        else:
            raise ValueError("phase must be tm.MissionPhase.TOM or tm.MissionPhase.INT")
        return result

    def orbits_per_swath(self) -> int:
        if self.mission_phase == tm.MissionPhase.TOM:
            result = TOM_ORBITS_PER_SWATH
        elif self.mission_phase == tm.MissionPhase.INT:
            result = INT_ORBITS_PER_SWATH
        else:
            raise ValueError("phase must be tm.MissionPhase.TOM or tm.MissionPhase.INT")
        return result
    
    def orbits_per_gc(self) -> int:
        if self.mission_phase == tm.MissionPhase.TOM:
            result = TOM_ORBITS_PER_GC
        elif self.mission_phase == tm.MissionPhase.INT:
            result = INT_ORBITS_PER_GC
        else:
            raise ValueError("phase must be tm.MissionPhase.TOM or tm.MissionPhase.INT")
        return result
    
    def verify_orbit(self):
        if self.orbit < 1:
            raise ValueError("orbit must be > 0")
            
    def gcid(self) -> int:
        self.verify_orbit()
        return int(np.ceil(self.orbit / self.orbits_per_gc()))

    def mcid(self) -> int:
        self.verify_orbit()
        orbit = self.orbit - (self.gcid() - 1) * self.orbits_per_gc()
        result = int(np.ceil(orbit / self.orbits_per_mc()))
        if result > 7:
            raise ValueError("computed MCID is greater than 7")
        return result
    
    def swathid(self) -> int:
        self.verify_orbit()
        orbit = self.orbit - (self.gcid() - 1) * self.orbits_per_gc() - (self.mcid() - 1) * self.orbits_per_mc()
        swaths = SWATHS if self.mcid() < 7 else 1
        if orbit < self.orbits_per_swath() * swaths:
            result = int(np.ceil(orbit / self.orbits_per_swath()))
        else:
            # SRM
            result = 0
        return result
    
    def rcid(self) -> int:
        self.verify_orbit()
        swathid = self.swathid()
        if swathid == 0:
            # SRM
            result = 0
        else:
            orbit = self.orbit - (self.gcid() - 1) * self.orbits_per_gc() - (self.mcid() - 1) * self.orbits_per_mc() - \
                (swathid - 1) * self.orbits_per_swath()
            result = int(np.ceil(orbit / ORBITS_PER_RC))
        return result
    
    def track(self) -> int:
        self.verify_orbit()
        swathid = self.swathid()
        if swathid == 0:
            # SRM
            result = 0
        else:
            result = self.orbit - (self.gcid() - 1) * self.orbits_per_gc() - (self.mcid() - 1) * self.orbits_per_mc() - \
                (swathid - 1) * self.orbits_per_swath() - (self.rcid() - 1) * ORBITS_PER_RC
        return result
    
    def to_interval(self) -> pd.Interval:
        if None in (self.start, self.stop):
            raise TypeError("orbit's start and stop times must be not None")
        return pd.Interval(self.start, self.stop, closed='both')


@dataclass
class Grid:
    
    orbit_start: pd.Timestamp
    orbit_stop: pd.Timestamp
    duration: float
    initial_overlap: float
    final_overlap: float
    num_intervals: int
        
    def interval(self, n: int) -> pd.Interval:
        if n < 1 or n > self.num_intervals:
            raise ValueError(f'interval number must be included in [0, {self.num_intervals}]')
        left = self.orbit_start + pd.to_timedelta((n - 1) * self.duration - self.initial_overlap, 's')
        if n == self.num_intervals:
            right = self.orbit_stop + pd.to_timedelta(self.final_overlap, 's')
        else:
            right = self.orbit_start + pd.to_timedelta(n * self.duration + self.final_overlap, 's')
        return pd.Interval(left, right, closed='both')
    
    def intervals(self, restriction: Optional[pd.Interval] = None,
                  min_duration: Optional[float] = None) -> list[tuple[int, pd.Interval]]:
        result = []
        for n in range(self.num_intervals):
            i = self.interval(n + 1)
            if restriction is None:
                result.append((n + 1, i))
            else:
                if i.overlaps(restriction):
                    start = max(restriction.left, i.left)
                    stop = min(restriction.right, i.right)
                    result.append((n + 1, pd.Interval(start, stop, closed='both')))
        if min_duration is not None:
            _result = []
            for k in range(len(result)):
                if result[k][1].length.total_seconds() >= min_duration:
                    start = result[k][1].left
                    stop = result[k][1].right
                    if k - 1 >= 0 and result[k - 1][1].length.total_seconds() < min_duration:
                        start = result[k - 1][1].left
                    if k + 1 < len(result) and result[k + 1][1].length.total_seconds() < min_duration:
                        stop = result[k + 1][1].right
                    _result.append((result[k][0], pd.Interval(start, stop, closed='both')))
            result = _result
        return result

    
@dataclass
class SliceGrid(Grid):

    orbit_start: pd.Timestamp
    orbit_stop: pd.Timestamp
    duration: float = const.SLICE_GRID_DURATION
    initial_overlap: float = const.SLICE_INITIAL_OVERLAP
    final_overlap: float = const.SLICE_FINAL_OVERLAP
    num_intervals: int = const.NUM_SLICES

        
@dataclass
class FrameGrid(Grid):

    orbit_start: pd.Timestamp
    orbit_stop: pd.Timestamp
    duration: float = const.FRAME_GRID_DURATION
    initial_overlap: float = const.FRAME_INITIAL_OVERLAP
    final_overlap: float = const.FRAME_FINAL_OVERLAP
    num_intervals: int = const.NUM_FRAMES

        
def get_orbits(simulation: pd.DataFrame, idx: int) -> list[pd.Interval]:
    """Get list of orbit intervals overlapping acuisition identified by `idx`."""
    result = [
        pd.Interval(simulation.loc[idx]['Anx'], simulation.loc[idx]['NextAnx'], closed='both'),
    ]
    if simulation.loc[idx]['End'] > simulation.loc[idx]['NextAnx']:
        result.append(
            pd.Interval(
                simulation[simulation['Orbit'] == simulation.loc[idx]['Orbit'] + 1]['Anx'].min(),
                simulation[simulation['Orbit'] == simulation.loc[idx]['Orbit'] + 1]['NextAnx'].min(),
                closed='both'
            )
        )
    return result


def get_slices(orbits: list[pd.Interval], interval: pd.Interval, min_duration: Optional[float] = None) -> pd.DataFrame:
    """Get list of slices having overlap with `interval`."""
    slices = []
    for orbit in orbits:
        sg = SliceGrid(orbit.left, orbit.right)
        slices.extend(sg.intervals(interval, min_duration))  
    return pd.DataFrame(
        [{'n': s[0], 'start': s[1].left, 'stop': s[1].right, 'duration': s[1].length.total_seconds()}
         for s in slices]
    )


def get_frames(orbits: list[pd.Interval], interval: pd.Interval, min_duration: Optional[float] = None) -> pd.DataFrame:
    """Get list of frames having overlap with `interval`."""
    frames = []
    for orbit in orbits:
        fg = FrameGrid(orbit.left, orbit.right)
        frames.extend(fg.intervals(interval, min_duration))
    return pd.DataFrame(
        [{'n': f[0], 'start': f[1].left, 'stop': f[1].right, 'duration': f[1].length.total_seconds()}
         for f in frames]
    )


def raw_generation_scenario(tds_name: str, simulation: pd.DataFrame, idx: int, mode: str) -> dict:
    if mode == 'MM':
        pids = [25, 26]
    elif mode == 'RO':
        pids = [27, 28]
    elif mode == 'EC':
        pids = [35, 36]
    else:
        raise ValueError(f"unknown mode: {mode}")
    
    data_take = pd.Interval(simulation.loc[idx]['Start'], simulation.loc[idx]['End'], closed='both')
    plt_data_validity = pd.Interval(
        data_take.left - pd.to_timedelta(const.PLATFORM_ANCILLARY_INITIAL_MARGIN, 's'),
        data_take.right + pd.to_timedelta(const.PLATFORM_ANCILLARY_FINAL_MARGIN, 's'),
        closed='both'
    )
    acquisition_date = data_take.right + pd.to_timedelta(30, 'm')
    return {
        'name': f"{tds_name} - DTID {simulation.loc[idx]['DTID']}",
        'file_name': 'N/A',
        'processor_name': 'procsim',
        'processor_version': procsim_version,
        'task_name': 'N/A',
        'task_version': 'N/A',
        'log_level': 'debug',
        'begin_position': data_take.left.isoformat() + 'Z',
        'end_position': data_take.right.isoformat() + 'Z',
        'acquisition_date': acquisition_date.isoformat() + 'Z',
        'acquisition_station': 'SV',
        'baseline': 1,
        'outputs': [
            {
                'type': 'RAW_022_10',
                'begin_position': plt_data_validity.left.isoformat() + 'Z',
                'end_position': plt_data_validity.right.isoformat() + 'Z',
                'num_isp': int(np.ceil(ANC_ISPS_PER_SEC * plt_data_validity.length.total_seconds())),
                'num_isp_erroneous': 0,
                'num_isp_corrupt': 0
            }
        ] + [
            {
                'type': f'RAW_{pid:03d}_10',
                'num_isp': int(np.ceil(ANC_ISPS_PER_SEC * data_take.length.total_seconds())),
                'num_isp_erroneous': 0,
                'num_isp_corrupt': 0
            }
            for pid in [23, 24]
        ] + [
            {
                'type': f'RAW_{pid:03d}_10',
                'num_isp': int(np.ceil(SAR_ISPS_PER_SEC * data_take.length.total_seconds())),
                'num_isp_erroneous': 0,
                'num_isp_corrupt': 0
            }
            for pid in pids
        ]
    }


def l0pfs1_processing_scenarios(tds_name: str, simulation: pd.DataFrame, idx: int, mode: str, phase: str) -> list[dict]:
    pids = [22, 23, 24]
    if mode == 'MM':
        pids += [25, 26]
    elif mode == 'RO':
        pids += [27, 28]
    elif mode == 'EC':
        pids += [35, 36]
    else:
        raise ValueError(f"unknown mode: {mode}")

    data_take = pd.Interval(simulation.loc[idx]['Start'], simulation.loc[idx]['End'], closed='both')
    baseline = 1
    orbits = get_orbits(simulation, idx)
    result = [
        {
            'name': f"{tds_name} - {mode} - DTID {simulation.loc[idx]['DTID']} - L0PFS1 - RAW_{pid:03d}_10",
            'file_name': 'l0pfs1.sh',
            'processor_name': f'L0PFS1{pid}',
            'processor_version': '01.00',
            'task_name': f'L0PFS1{pid}',
            'task_version': '01.00',
            'log_level': 'debug',
            'baseline': baseline,
            'anx': [o.left.isoformat() + 'Z' for o in orbits],
            'outputs': [
                {
                    'type': f'RAWS{pid:03d}_10',
                    'metadata_source': f'.*RAW_{pid:03d}_10_.*'
                }
            ]
        }
        for pid in pids
    ]
    return result


def l0pfs2_processing_scenario(tds_name: str, simulation: pd.DataFrame, idx: int, mode: str, phase: str) -> dict:
    if mode == 'MM':
        outputs = [
            {
                'type': f"{simulation.loc[idx]['SensorMode'][-2:]}_RAW__0S",
                'metadata_source': '.*RAWS025_10_.*'
            },
            {
                'type': f"{simulation.loc[idx]['SensorMode'][-2:]}_RAWP_0M",
                'metadata_source': '.*RAWS025_10_.*'
            }            
        ]
    elif mode == 'RO':
        outputs = [
            {
                'type': "RO_RAW__0S",
                'metadata_source': '.*RAWS027_10_.*'
            }
        ]
    elif mode == 'EC':
            {
                'type': "EC_RAWP_0S",
                'metadata_source': '.*RAWS035_10_.*'
            },
            {
                'type': "EC_RAWP_0M",
                'metadata_source': '.*RAWS035_10_.*'
            }            
    else:
        raise ValueError(f"unknown mode: {mode}")
    data_take = pd.Interval(simulation.loc[idx]['Start'], simulation.loc[idx]['End'], closed='both')
    baseline = 1
    orbits = get_orbits(simulation, idx)
    result = {
        'name': f"{tds_name} - {mode} - DTID {simulation.loc[idx]['DTID']} - L0PFS2",
        'file_name': 'l0pfs2.sh',
        'processor_name': 'L0PFS2',
        'processor_version': '01.00',
        'task_name': 'L0PFS2',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': baseline,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(simulation.loc[idx]['DTID']),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'swath': simulation.loc[idx]['SensorMode'][-2:],
                'operational_mode': mode
            }
        ],
        'mission_phase': phase,
        'global_coverage_id': str(simulation.loc[idx]['gcid']),
        'major_cycle_id': str(simulation.loc[idx]['mcid']),
        'repeat_cycle_id': str(simulation.loc[idx]['rcid']),
        'track_nr': str(simulation.loc[idx]['track']),
        'num_l0_lines': str(int(np.ceil(SAR_ISPS_PER_SEC * const.SLICE_DURATION))),
        'num_l0_lines_corrupt': '0',
        'num_l0_lines_missing': '0',
        'outputs': outputs
    }
    return result


def l0pfs3_processing_scenario(tds_name: str, simulation: pd.DataFrame, idx: int, mode: str, phase: str) -> dict:
    if mode == 'MM':
        swath_id = simulation.loc[idx]['SensorMode'][-2:]
    elif mode == 'EC':
        swath_id = 'RO'
    else:
        raise ValueError(f"unknown mode: {mode}")
    data_take = pd.Interval(simulation.loc[idx]['Start'], simulation.loc[idx]['End'], closed='both')
    baseline = 1
    orbits = get_orbits(simulation, idx)
    result = {
        'name': f"{tds_name} - {mode} - DTID {simulation.loc[idx]['DTID']} - L0PFS3",
        'file_name': 'l0pfs3.sh',
        'processor_name': 'L0PFS3',
        'processor_version': '01.00',
        'task_name': 'L0PFS3',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': baseline,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(simulation.loc[idx]['DTID']),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'swath': swath_id,
                'operational_mode': mode
            }
        ],
        'mission_phase': phase,
        'global_coverage_id': str(simulation.loc[idx]['gcid']),
        'major_cycle_id': str(simulation.loc[idx]['mcid']),
        'repeat_cycle_id': str(simulation.loc[idx]['rcid']),
        'track_nr': str(simulation.loc[idx]['track']),
        'num_l0_lines': str(int(np.ceil(SAR_ISPS_PER_SEC * const.SLICE_DURATION))),
        'num_l0_lines_corrupt': '0',
        'num_l0_lines_missing': '0',
        'outputs': [
            {
                'file_type': f'{swath_id}_RAW__0M',
                #'begin_position': data_take.left.isoformat() + 'Z',
                #'end_position': data_take.right.isoformat() + 'Z',                
            }
        ]
    }
    return result


def l0pfs4_processing_scenario(tds_name: str, simulation: pd.DataFrame, idx: int, mode: str, phase: str) -> dict:
    data_take = pd.Interval(simulation.loc[idx]['Start'], simulation.loc[idx]['End'], closed='both')
    baseline = 1
    orbits = get_orbits(simulation, idx)
    result = {
        'name': f"{tds_name} - {mode} - DTID {simulation.loc[idx]['DTID']} - L0PFS4",
        'file_name': 'l0pfs4.sh',
        'processor_name': 'L0PFS4',
        'processor_version': '01.00',
        'task_name': 'L0PFS4',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': baseline,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(simulation.loc[idx]['DTID']),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'swath': simulation.loc[idx]['SensorMode'][-2:],
                'operational_mode': mode
            }
        ],
        'mission_phase': phase,
        'global_coverage_id': str(simulation.loc[idx]['gcid']),
        'major_cycle_id': str(simulation.loc[idx]['mcid']),
        'repeat_cycle_id': str(simulation.loc[idx]['rcid']),
        'track_nr': str(simulation.loc[idx]['track']),
        'num_l0_lines': str(int(np.ceil(ANC_ISPS_PER_SEC * const.SLICE_DURATION))),
        'num_l0_lines_corrupt': '0',
        'num_l0_lines_missing': '0',
        'outputs': [
            {
                'file_type': 'AC_RAW__0A',
                #'begin_position': data_take.left.isoformat() + 'Z',
                #'end_position': data_take.right.isoformat() + 'Z',
                'leading_margin': const.PLATFORM_ANCILLARY_INITIAL_MARGIN,
                'trailing_margin': const.PLATFORM_ANCILLARY_FINAL_MARGIN
            }
        ]
    }
    return result


def l0pfs5_processing_scenario(tds_name: str, simulation: pd.DataFrame, idx: int, phase: str) -> dict:
    data_take = pd.Interval(simulation.loc[idx]['Start'], simulation.loc[idx]['End'], closed='both')
    baseline = 1
    orbits = get_orbits(simulation, idx)
    result = {
        'name': f"{tds_name} - EC - DTID {simulation.loc[idx]['DTID']} - L0PFS5",
        'file_name': 'l0pfs5.sh',
        'processor_name': 'L0PFS5',
        'processor_version': '01.00',
        'task_name': 'L0PFS5',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': baseline,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(simulation.loc[idx]['DTID']),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'swath': 'EC',
                'operational_mode': 'EC'
            }
        ],
        'mission_phase': phase,
        'global_coverage_id': str(simulation.loc[idx]['gcid']),
        'major_cycle_id': str(simulation.loc[idx]['mcid']),
        'repeat_cycle_id': str(simulation.loc[idx]['rcid']),
        'track_nr': str(simulation.loc[idx]['track']),
        'num_l0_lines': str(int(np.ceil(SAR_ISPS_PER_SEC * const.SLICE_DURATION))),
        'num_l0_lines_corrupt': '0',
        'num_l0_lines_missing': '0',
        'outputs': [
            {
                'file_type': 'EC_RAW__0S',
                #'begin_position': data_take.left.isoformat() + 'Z',
                #'end_position': data_take.right.isoformat() + 'Z',
            }
        ]
    }
    return result


def oapf_processing_scenario(tds_name: str, simulation: pd.DataFrame, idx: int, mode: str, phase: str) -> dict:
    data_take = pd.Interval(simulation.loc[idx]['Start'], simulation.loc[idx]['End'], closed='both')
    baseline = 1
    orbits = get_orbits(simulation, idx)
    result = {
        'name': f"{tds_name} - {mode} - DTID {simulation.loc[idx]['DTID']} - OAPF",
        'file_name': 'oapf.sh',
        'processor_name': 'OAPF',
        'processor_version': '01.00',
        'task_name': 'OAPF',
        'task_version': '01.00',
        'log_level': 'debug',
        'baseline': baseline,
        'anx': [o.left.isoformat() + 'Z' for o in orbits],
        'data_takes': [
            {
                'data_take_id': str(simulation.loc[idx]['DTID']),
                'start': data_take.left.isoformat() + 'Z',
                'stop': data_take.right.isoformat() + 'Z',
                'operational_mode': mode
            }
        ],
        'outputs': [
            {
                'file_type': 'AUX_ATT___',
                #'begin_position': data_take.left.isoformat() + 'Z',
                #'end_position': data_take.right.isoformat() + 'Z',
                'leading_margin': const.PLATFORM_ANCILLARY_INITIAL_MARGIN,
                'trailing_margin': const.PLATFORM_ANCILLARY_FINAL_MARGIN
            },
            {
                'file_type': 'AUX_ORB___',
                #'begin_position': data_take.left.isoformat() + 'Z',
                #'end_position': data_take.right.isoformat() + 'Z',
                'leading_margin': const.PLATFORM_ANCILLARY_INITIAL_MARGIN,
                'trailing_margin': const.PLATFORM_ANCILLARY_FINAL_MARGIN
            }
        ]
    }
    return result


def l1_processing_scenarios(tds_name: str, simulation: pd.DataFrame, idx: int, phase: str) -> dict:
    data_take = pd.Interval(simulation.loc[idx]['Start'], simulation.loc[idx]['End'], closed='both')
    orbit = pd.Interval(simulation.loc[idx]['Anx'], simulation.loc[idx]['NextAnx'])
    slices = get_slices([orbit], data_take, SLICE_MIN_DURATION)
    swath_id = simulation.loc[idx]['SensorMode'][-2:]
    dt_polygon = simulation.loc[idx, ['NW_Lat', 'NW_Lon', 'NE_Lat', 'NE_Lon', 'SE_Lat', 'SE_Lon', 'SW_Lat', 'SW_Lon', 'NW_Lat', 'NW_Lon']].to_numpy()
    footprint_polygon = ' '.join(dt_polygon.astype(str))
    baseline = 1
    result = [
        {
            'name': f"{tds_name} - DTID {simulation.loc[idx]['DTID']} - L1PF - slice {slice_number}",
            'file_name': 'l1pf.sh',
            'processor_name': 'L1PF',
            'processor_version': '01.00',
            'task_name': 'L1PF',
            'task_version': '01.00',
            'log_level': 'debug',
            'baseline': baseline,
            'anx': [o.left.isoformat() + 'Z' for o in orbits],
            'data_takes': [
                {
                    'data_take_id': str(simulation.loc[idx]['DTID']),
                    'start': data_take.left.isoformat() + 'Z',
                    'stop': data_take.right.isoformat() + 'Z',
                    'swath': swath_id,
                    'operational_mode': 'SM'
                }
            ],
            'mission_phase': phase,
            'global_coverage_id': str(simulation.loc[idx]['gcid']),
            'major_cycle_id': str(simulation.loc[idx]['mcid']),
            'repeat_cycle_id': str(simulation.loc[idx]['rcid']),
            'track_nr': str(simulation.loc[idx]['track']),
            'footprint_polygon': footprint_polygon,
            'outputs': [
                {
                    'file_type': f'{swath_id}_SCS__1S',
                    #'begin_position': slices.query('n == @slice_number')['start'].iloc[0].isoformat() + 'Z',
                    #'end_position': slices.query('n == @slice_number')['stop'].iloc[0].isoformat() + 'Z'
                },
                {
                    'file_type': f'{swath_id}_SCS__1M',
                    #'begin_position': slices.query('n == @slice_number')['start'].iloc[0].isoformat() + 'Z',
                    #'end_position': slices.query('n == @slice_number')['stop'].iloc[0].isoformat() + 'Z'
                },
                {
                    'file_type': f'{swath_id}_DGM__1S',
                    #'begin_position': slices.query('n == @slice_number')['start'].iloc[0].isoformat() + 'Z',
                    #'end_position': slices.query('n == @slice_number')['stop'].iloc[0].isoformat() + 'Z'
                }
            ]
        }
        for slice_number in slices['n']
    ]
    return result


def l1c_processing_scenarios(tds_name: str, simulation: pd.DataFrame, idx: int, phase: str) -> dict:
    data_take = pd.Interval(simulation.loc[idx]['Start'], simulation.loc[idx]['End'], closed='both')
    orbit = pd.Interval(simulation.loc[idx]['Anx'], simulation.loc[idx]['NextAnx'])
    frames = get_frames([orbit], data_take, FRAME_MIN_DURATION)
    swath_id = simulation.loc[idx]['SensorMode'][-2:]
    dt_polygon = simulation.loc[idx, ['NW_Lat', 'NW_Lon', 'NE_Lat', 'NE_Lon', 'SE_Lat', 'SE_Lon', 'SW_Lat', 'SW_Lon', 'NW_Lat', 'NW_Lon']].to_numpy()
    footprint_polygon = ' '.join(dt_polygon.astype(str))
    baseline = 1
    result = [
        {
            'name': f"{tds_name} - DTID {simulation.loc[idx]['DTID']} - L1PF - frame {frame_number}",
            'file_name': 'stackpf.sh',
            'processor_name': 'STACK',
            'processor_version': '01.00',
            'task_name': 'STACK',
            'task_version': '01.00',
            'log_level': 'debug',
            'baseline': baseline,
            'anx': [o.left.isoformat() + 'Z' for o in orbits],
            'data_takes': [
                {
                    'data_take_id': str(simulation.loc[idx]['DTID']),
                    'start': data_take.left.isoformat() + 'Z',
                    'stop': data_take.right.isoformat() + 'Z',
                    'swath': swath_id,
                    'operational_mode': 'SM'
                }
            ],
            'mission_phase': phase,
            'global_coverage_id': str(simulation.loc[idx]['gcid']),
            'major_cycle_id': str(simulation.loc[idx]['mcid']),
            'repeat_cycle_id': str(simulation.loc[idx]['rcid']),
            'track_nr': str(simulation.loc[idx]['track']),
            'footprint_polygon': footprint_polygon,
            'outputs': [
                {
                    'file_type': f'{swath_id}_STA__1S',
                    #'begin_position': slices.query('n == @slice_number')['start'].iloc[0].isoformat() + 'Z',
                    #'end_position': slices.query('n == @slice_number')['stop'].iloc[0].isoformat() + 'Z'
                },
                {
                    'file_type': f'{swath_id}_STA__1M',
                    #'begin_position': slices.query('n == @slice_number')['start'].iloc[0].isoformat() + 'Z',
                    #'end_position': slices.query('n == @slice_number')['stop'].iloc[0].isoformat() + 'Z'
                }
            ]
        }
        for frame_number in frames['n']
    ]
    return result


%reload_ext watermark
%watermark -iv

biotm : 1.0a0
numpy : 1.20.2
json  : 2.0.9
pandas: 1.2.5



## TOM_TDS

### Load data from SaVoir simulation

In [2]:
def next_anx(row):
    next_orbit = row['Orbit'] + 1
    next_anx = row['Anx'] + pd.to_timedelta(ORBIT_DURATION, 's')
    r = simulation.query("Orbit == @next_orbit")
    if r.shape[0] > 0:
        next_anx = r['Anx'].min()
    return next_anx

simulation = pd.read_excel('Export_TOM_Full_GC_S1_S2_S3_RO_S1SRM.xlsx')

# add DTID
simulation['DTID'] = [int(tm.DataTakeID(row.Orbit, row.TAnx).tobytes().hex(), 16) for row in simulation.itertuples()]
# add mcid, rcid, track, ext anx
orbits = [AbsoluteOrbit(n, tm.MissionPhase.INT) for n in simulation['Orbit']]
simulation['gcid'] = [o.gcid() for o in orbits]
simulation['mcid'] = [o.mcid() for o in orbits]
simulation['rcid'] = [o.rcid() for o in orbits]
simulation['track'] = [o.track() for o in orbits]
simulation['NextAnx'] = simulation.apply(next_anx, axis=1)

simulation.sort_values(by=['Start'], inplace=True)

simulation.columns

Index(['Start', 'End', 'Duration', 'Satellite', 'SensorMode', 'Width',
       'Length', 'SwathArea', 'Pass', 'NW_Lat', 'NW_Lon', 'NE_Lat', 'NE_Lon',
       'SE_Lat', 'SE_Lon', 'SW_Lat', 'SW_Lon', 'Orbit', 'Anx', 'AnxLon',
       'TAnx', 'DTID', 'gcid', 'mcid', 'rcid', 'track', 'NextAnx'],
      dtype='object')

### MM scenario

#### Data Take selection

In [3]:
simulation.query("SensorMode != 'Receive Only'")[['Anx', 'Start', 'End', 'NextAnx', 'Duration', 'SensorMode']].head(10)

,Anx,Start,End,NextAnx,Duration,SensorMode
0,2017-01-01 06:00:01.272,2017-01-01 06:01:31.394,2017-01-01 06:03:44.504,2017-01-01 07:38:12.217,133.110305,Stripmap S1
2,2017-01-01 06:00:01.272,2017-01-01 06:52:49.307,2017-01-01 06:55:03.719,2017-01-01 07:38:12.217,134.411778,Stripmap S1
4,2017-01-01 07:38:12.217,2017-01-01 08:28:10.480,2017-01-01 08:29:15.351,2017-01-01 09:16:23.162,64.870512,Stripmap S1
5,2017-01-01 07:38:12.217,2017-01-01 08:30:20.767,2017-01-01 08:36:51.226,2017-01-01 09:16:23.162,390.458948,Stripmap S1
6,2017-01-01 07:38:12.217,2017-01-01 09:09:51.641,2017-01-01 09:18:02.670,2017-01-01 09:16:23.162,491.029120,Stripmap S1
8,2017-01-01 09:16:23.162,2017-01-01 09:46:40.936,2017-01-01 09:49:12.859,2017-01-01 10:54:34.108,151.923140,Stripmap S1
9,2017-01-01 09:16:23.162,2017-01-01 09:50:14.321,2017-01-01 09:53:27.219,2017-01-01 10:54:34.108,192.897880,Stripmap S1
10,2017-01-01 09:16:23.162,2017-01-01 09:55:51.150,2017-01-01 09:56:23.150,2017-01-01 10:54:34.108,32.000000,Stripmap S1
11,2017-01-01 09:16:23.162,2017-01-01 10:01:27.997,2017-01-01 10:03:21.433,2017-01-01 10:54:34.108,113.436128,Stripmap S1
12,2017-01-01 09:16:23.162,2017-01-01 10:04:57.070,2017-01-01 10:08:08.894,2017-01-01 10:54:34.108,191.824403,Stripmap S1


ID of selected acquisition: 0

In [4]:
acq_idx = 0

In [5]:
data_take = pd.Interval(simulation.loc[acq_idx, 'Start'], simulation.loc[acq_idx, 'End'], closed='both')
data_take

Interval('2017-01-01 06:01:31.394000', '2017-01-01 06:03:44.504000', closed='both')

In [6]:
dtid = simulation.loc[acq_idx, 'DTID']
dtid

8282

#### Swath

In [7]:
# get swath id from SensorMode name
swath_id = simulation.loc[acq_idx]['SensorMode'][-2:]
swath_id

'S1'

In [8]:
# crosscheck with orbit number
AbsoluteOrbit(simulation.loc[acq_idx]['Orbit'], tm.MissionPhase.INT).swathid()

1

#### Global coverage id, major cycle id, repeat cycle id, drift flag, orbit and track

In [9]:
gcid = simulation.loc[acq_idx]['gcid']
mcid = simulation.loc[acq_idx]['mcid']
rcid = simulation.loc[acq_idx]['rcid']
track = simulation.loc[acq_idx]['track']

In [10]:
dt_polygon = simulation.loc[acq_idx, ['NW_Lat', 'NW_Lon', 'NE_Lat', 'NE_Lon', 'SE_Lat', 'SE_Lon', 'SW_Lat', 'SW_Lon', 'NW_Lat', 'NW_Lon']].to_numpy()

footprint_polygon = ' '.join(dt_polygon.astype(str))

#### Acquisition

Acquisition date is set to 30 minutes after data take stop

In [11]:
acquisition_date = data_take.right + pd.to_timedelta(30, 'm')
acquisition_station = 'SV'  # Svalbard
acquisition_date # start

Timestamp('2017-01-01 06:33:44.504000')

#### Affected Orbits

In [12]:
orbits = get_orbits(simulation, acq_idx)
orbits

[Interval('2017-01-01 06:00:01.272000', '2017-01-01 07:38:12.217000', closed='both')]

#### Level 0 slices

In [13]:
slices = get_slices(orbits, data_take, SLICE_MIN_DURATION)
slices

,n,start,stop,duration
0,2,2017-01-01 06:01:31.394000000,2017-01-01 06:03:18.303615480,106.909615
1,3,2017-01-01 06:03:06.303615480,2017-01-01 06:03:44.504000000,38.200384


In case of platform ancillary data, validity starts 16 seconds before data take start:

In [14]:
plt_data_validity = pd.Interval(
    data_take.left - pd.to_timedelta(const.PLATFORM_ANCILLARY_INITIAL_MARGIN, 's'),
    data_take.right + pd.to_timedelta(const.PLATFORM_ANCILLARY_FINAL_MARGIN, 's'),
    closed='both'
)
plt_slices = get_slices(orbits, plt_data_validity, SLICE_MIN_DURATION)
plt_slices

,n,start,stop,duration
0,2,2017-01-01 06:01:15.394000000,2017-01-01 06:03:18.303615480,122.909615
1,3,2017-01-01 06:03:06.303615480,2017-01-01 06:03:44.504000000,38.200384


#### Level 1 frames

In [15]:
frames = get_frames(orbits, data_take, FRAME_MIN_DURATION)
frames

,n,start,stop,duration
0,6,2017-01-01 06:01:31.394000000,2017-01-01 06:01:57.290969288,25.896969
1,7,2017-01-01 06:01:55.290969288,2017-01-01 06:02:16.294130836,21.003161
2,8,2017-01-01 06:02:14.294130836,2017-01-01 06:02:35.297292384,21.003161
3,9,2017-01-01 06:02:33.297292384,2017-01-01 06:02:54.300453932,21.003161
4,10,2017-01-01 06:02:52.300453932,2017-01-01 06:03:13.303615480,21.003161
5,11,2017-01-01 06:03:11.303615480,2017-01-01 06:03:32.306777028,21.003161
6,12,2017-01-01 06:03:30.306777028,2017-01-01 06:03:44.504000000,14.197222


#### Additional data takes for stack

Select additional data takes having the same global coverage id, major cycle id, swath (sensor mode) and track:

In [16]:
def overlap(data_take_start, data_take_stop, orbit_start, orbit_stop, frame_numbers):
    data_take = pd.Interval(data_take_start, data_take_stop, closed='both')
    frame_grid = FrameGrid(orbit_start, orbit_stop)
    for frame_number in frame_numbers:
        overlap = frame_grid.interval(frame_number).overlaps(data_take)
        if overlap is False:
            return False
    return True

sensor_mode = simulation.loc[acq_idx]['SensorMode']
track = simulation.loc[acq_idx]['track']
result = simulation.query("gcid == @gcid and mcid == @mcid and SensorMode == @sensor_mode and track == @track")
# Select only data takes having overlaps with frame numbers selected above
idx = result.apply(
    lambda r: overlap(r['Start'], r['End'], r['Anx'], r['NextAnx'], frames['n']),
    axis=1
)
result[idx]

,Start,End,Duration,Satellite,SensorMode,Width,Length,SwathArea,Pass,NW_Lat,...,Orbit,Anx,AnxLon,TAnx,DTID,gcid,mcid,rcid,track,NextAnx
0,2017-01-01 06:01:31.394,2017-01-01 06:03:44.504,133.110305,BIOMASS,Stripmap S1,53.961516,915.099690,49580.620718,ASCENDING,13.001162,...,1,2017-01-01 06:00:01.272,1.849826e-07,90.122152,8282,1,1,1,1,2017-01-01 07:38:12.217
203,2017-01-04 06:01:33.020,2017-01-04 06:03:46.097,133.076316,BIOMASS,Stripmap S1,53.961496,914.866109,49570.851246,ASCENDING,13.001108,...,45,2017-01-04 06:00:02.866,-6.647473e-03,90.154524,368730,1,1,2,1,2017-01-04 07:38:13.811
406,2017-01-07 06:01:34.646,2017-01-07 06:03:47.690,133.043820,BIOMASS,Stripmap S1,53.961496,914.642788,49558.247959,ASCENDING,13.001127,...,89,2017-01-07 06:00:04.459,-1.329131e-02,90.186905,729178,1,1,3,1,2017-01-07 07:38:15.404
609,2017-01-10 06:01:36.271,2017-01-10 06:03:49.285,133.013277,BIOMASS,Stripmap S1,53.961495,914.432883,49544.842820,ASCENDING,13.001220,...,133,2017-01-10 06:00:06.052,-1.992187e-02,90.219591,1089626,1,1,1,1,2017-01-10 07:38:16.997
812,2017-01-13 06:01:37.899,2017-01-13 06:03:50.878,132.979137,BIOMASS,Stripmap S1,53.961496,914.198236,49534.279940,ASCENDING,13.001225,...,177,2017-01-13 06:00:07.646,-2.657767e-02,90.252429,1450074,1,1,2,1,2017-01-13 07:38:18.591
1015,2017-01-16 06:01:39.523,2017-01-16 06:03:52.470,132.946797,BIOMASS,Stripmap S1,53.975102,913.975943,49519.171485,ASCENDING,13.001162,...,221,2017-01-16 06:00:09.237,-3.319050e-02,90.285979,1810522,1,1,3,1,2017-01-16 07:38:20.182
1218,2017-01-19 06:01:41.149,2017-01-19 06:03:54.064,132.914460,BIOMASS,Stripmap S1,53.975102,913.753669,49508.257056,ASCENDING,13.001202,...,265,2017-01-19 06:00:10.831,-3.983360e-02,90.318618,2170970,1,1,1,1,2017-01-19 07:38:21.776


In [17]:
idxs = list(result[idx].index.to_numpy())
idxs

[0, 203, 406, 609, 812, 1015, 1218]

#### Raw Data generation

Scenario generation

In [18]:
config = {
    'mission': 'biomass',
    'scenarios': [raw_generation_scenario('TOM_TDS', simulation, idx, 'MM') for idx in idxs]
}
with open('tom-tds-raw-gen-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)

#### Level 0

Scenario generation

In [19]:
config = {
    'mission': 'biomass',
    'scenarios': []
}
for idx in idxs:
    config['scenarios'].extend(l0pfs1_processing_scenarios('TOM_TDS', simulation, idx, 'MM', 'TOMOGRAPHIC'))
    config['scenarios'].append(l0pfs2_processing_scenario('TOM_TDS', simulation, idx, 'MM', 'TOMOGRAPHIC'))
    config['scenarios'].append(l0pfs3_processing_scenario('TOM_TDS', simulation, idx, 'MM', 'TOMOGRAPHIC'))
    config['scenarios'].append(l0pfs4_processing_scenario('TOM_TDS', simulation, idx, 'MM', 'TOMOGRAPHIC'))
    config['scenarios'].append(oapf_processing_scenario('TOM_TDS', simulation, idx, 'MM', 'TOMOGRAPHIC'))
with open('tom-tds-l0-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)

#### Level 1a/b

In [20]:
config = {
    'mission': 'biomass',
    'scenarios': []
}
for idx in idxs:
    config['scenarios'].extend(l1_processing_scenarios('TOM_TDS', simulation, idx, 'TOMOGRAPHIC'))
with open('tom-tds-l1-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)

#### Level 1c 

In [21]:
config = {
    'mission': 'biomass',
    'scenarios': []
}
for idx in idxs:
    config['scenarios'].extend(l1c_processing_scenarios('TOM_TDS', simulation, idx, 'TOMOGRAPHIC'))
with open('tom-tds-l1c-l2a-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)

### RO scenario

#### Data Take selection

In [22]:
columns = ['Anx', 'Start', 'End', 'NextAnx', 'Duration', 'gcid', 'mcid', 'rcid', 'Orbit', 'track']
simulation.query("SensorMode == 'Receive Only' and 100 < Duration < 150")[columns].head()

,Anx,Start,End,NextAnx,Duration,gcid,mcid,rcid,Orbit,track
14804,2017-10-18 15:37:54.823,2017-10-18 16:10:38.205,2017-10-18 16:12:55.049,2017-10-18 17:16:05.768,136.843692,2,2,1,4260,36
15006,2017-10-21 15:37:56.417,2017-10-21 16:10:39.347,2017-10-21 16:12:56.870,2017-10-21 17:16:07.362,137.523319,2,2,2,4304,36
15208,2017-10-24 15:37:58.009,2017-10-24 16:10:40.491,2017-10-24 16:12:58.691,2017-10-24 17:16:08.954,138.200432,2,2,3,4348,36
15410,2017-10-27 15:37:59.603,2017-10-27 16:10:41.636,2017-10-27 16:13:00.506,2017-10-27 17:16:10.548,138.869413,2,2,0,4392,0
15612,2017-10-30 15:38:01.195,2017-10-30 16:10:42.782,2017-10-30 16:13:02.326,2017-10-30 17:16:12.140,139.544101,2,2,0,4436,0


ID of selected acquisition: 14484

In [23]:
acq_idx = 14804

In [24]:
data_take = pd.Interval(simulation.loc[acq_idx, 'Start'], simulation.loc[acq_idx, 'End'], closed='both')
data_take

Interval('2017-10-18 16:10:38.205000', '2017-10-18 16:12:55.049000', closed='both')

In [25]:
dtid = simulation.loc[acq_idx, 'DTID']
dtid

34899883

#### Swath

In [26]:
swath_id = 'RO'

#### Global coverage id, major cycle id, repeat cycle id, drift flag, orbit and track

In [27]:
gcid = simulation.loc[acq_idx]['gcid']
mcid = simulation.loc[acq_idx]['mcid']
rcid = simulation.loc[acq_idx]['rcid']
track = simulation.loc[acq_idx]['track']

In [28]:
dt_polygon = simulation.loc[acq_idx, ['NW_Lat', 'NW_Lon', 'NE_Lat', 'NE_Lon', 'SE_Lat', 'SE_Lon', 'SW_Lat', 'SW_Lon', 'NW_Lat', 'NW_Lon']].to_numpy()

footprint_polygon = ' '.join(dt_polygon.astype(str))

#### Acquisition

Acquisition date is set to 30 minutes after data take stop

In [29]:
acquisition_date = data_take.right + pd.to_timedelta(30, 'm')
acquisition_station = 'SV'  # Svalbard
acquisition_date # start

Timestamp('2017-10-18 16:42:55.049000')

#### Affected Orbits

In [30]:
orbits = get_orbits(simulation, acq_idx)
orbits

[Interval('2017-10-18 15:37:54.823000', '2017-10-18 17:16:05.768000', closed='both')]

#### Level 0 slices

In [31]:
slices = get_slices(orbits, data_take, SLICE_MIN_DURATION)
slices

,n,start,stop,duration
0,21,2017-10-18 16:10:38.205000000,2017-10-18 16:11:17.154962540,38.949962
1,22,2017-10-18 16:11:05.154962540,2017-10-18 16:12:55.049000000,109.894037


In case of platform ancillary data, validity starts 16 seconds before data take start:

In [32]:
plt_data_validity = pd.Interval(
    data_take.left - pd.to_timedelta(const.PLATFORM_ANCILLARY_INITIAL_MARGIN, 's'),
    data_take.right + pd.to_timedelta(const.PLATFORM_ANCILLARY_FINAL_MARGIN, 's'),
    closed='both'
)
plt_slices = get_slices(orbits, plt_data_validity, SLICE_MIN_DURATION)
plt_slices

,n,start,stop,duration
0,21,2017-10-18 16:10:22.205000000,2017-10-18 16:11:17.154962540,54.949962
1,22,2017-10-18 16:11:05.154962540,2017-10-18 16:12:55.049000000,109.894037


#### Level 1 frames

In [33]:
frames = get_frames(orbits, data_take, FRAME_MIN_DURATION)
frames

,n,start,stop,duration
0,104,2017-10-18 16:10:38.205000000,2017-10-18 16:10:53.151800992,14.946800
1,105,2017-10-18 16:10:51.151800992,2017-10-18 16:11:12.154962540,21.003161
2,106,2017-10-18 16:11:10.154962540,2017-10-18 16:11:31.158124088,21.003161
3,107,2017-10-18 16:11:29.158124088,2017-10-18 16:11:50.161285636,21.003161
4,108,2017-10-18 16:11:48.161285636,2017-10-18 16:12:09.164447184,21.003161
5,109,2017-10-18 16:12:07.164447184,2017-10-18 16:12:28.167608732,21.003161
6,110,2017-10-18 16:12:26.167608732,2017-10-18 16:12:47.170770280,21.003161
7,111,2017-10-18 16:12:45.170770280,2017-10-18 16:12:55.049000000,9.878229


#### Raw Data generation

Scenario generation

In [34]:
with open('tom-tds-raw-gen-scenarios.json') as fh:
    config = json.load(fh)

config['scenarios'].append(raw_generation_scenario('TOM_TDS', simulation, acq_idx, 'RO'))

with open('tom-tds-raw-gen-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)

#### Level 0

Scenario generation

In [35]:
with open('tom-tds-l0-scenarios.json') as fh:
    config = json.load(fh)

config['scenarios'].extend(l0pfs1_processing_scenarios('TOM_TDS', simulation, acq_idx, 'RO', 'TOMOGRAPHIC'))
config['scenarios'].append(l0pfs2_processing_scenario('TOM_TDS', simulation, acq_idx, 'RO', 'TOMOGRAPHIC'))
config['scenarios'].append(l0pfs4_processing_scenario('TOM_TDS', simulation, acq_idx, 'RO', 'TOMOGRAPHIC'))
config['scenarios'].append(oapf_processing_scenario('TOM_TDS', simulation, acq_idx, 'RO', 'TOMOGRAPHIC'))

with open('tom-tds-l0-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)

## INT_TDS

### Load data from SaVoir simulation

In [36]:
def next_anx(row):
    next_orbit = row['Orbit'] + 1
    next_anx = row['Anx'] + pd.to_timedelta(ORBIT_DURATION, 's')
    r = simulation.query("Orbit == @next_orbit")
    if r.shape[0] > 0:
        next_anx = r['Anx'].min()
    return next_anx

simulation = pd.read_excel('Export_INT_Full_GC_S1_S2_S3_RO_S1SRM.xlsx')

# add DTID
simulation['DTID'] = [int(tm.DataTakeID(row.Orbit, row.TAnx).tobytes().hex(), 16) for row in simulation.itertuples()]
# add mcid, rcid, track, ext anx
orbits = [AbsoluteOrbit(n, tm.MissionPhase.INT) for n in simulation['Orbit']]
simulation['gcid'] = [o.gcid() for o in orbits]
simulation['mcid'] = [o.mcid() for o in orbits]
simulation['rcid'] = [o.rcid() for o in orbits]
simulation['track'] = [o.track() for o in orbits]
simulation['NextAnx'] = simulation.apply(next_anx, axis=1)

simulation.sort_values(by=['Start'], inplace=True)

simulation.columns

Index(['Start', 'End', 'Duration', 'Satellite', 'SensorMode', 'Width',
       'Length', 'SwathArea', 'Pass', 'NW_Lat', 'NW_Lon', 'NE_Lat', 'NE_Lon',
       'SE_Lat', 'SE_Lon', 'SW_Lat', 'SW_Lon', 'Orbit', 'Anx', 'AnxLon',
       'TAnx', 'DTID', 'gcid', 'mcid', 'rcid', 'track', 'NextAnx'],
      dtype='object')

### MM scenario

#### Data Take selection

Select Data Take across ANX

In [37]:
simulation.query("SensorMode != 'Receive Only'")[['Anx', 'Start', 'End', 'NextAnx', 'Duration', 'SensorMode']].head(10)

,Anx,Start,End,NextAnx,Duration,SensorMode
0,2017-01-01 06:00:01.272,2017-01-01 06:01:31.396,2017-01-01 06:03:44.506,2017-01-01 07:38:12.253,133.109331,Stripmap S1
2,2017-01-01 06:00:01.272,2017-01-01 06:52:49.335,2017-01-01 06:55:03.761,2017-01-01 07:38:12.253,134.425245,Stripmap S1
3,2017-01-01 06:00:01.272,2017-01-01 07:06:50.302,2017-01-01 07:20:24.262,2017-01-01 07:38:12.253,813.959573,Stripmap S1
5,2017-01-01 07:38:12.253,2017-01-01 08:28:10.551,2017-01-01 08:29:15.406,2017-01-01 09:16:23.235,64.854735,Stripmap S1
6,2017-01-01 07:38:12.253,2017-01-01 08:30:20.822,2017-01-01 08:36:51.284,2017-01-01 09:16:23.235,390.461360,Stripmap S1
7,2017-01-01 07:38:12.253,2017-01-01 08:45:01.315,2017-01-01 08:58:01.231,2017-01-01 09:16:23.235,779.916300,Stripmap S1
8,2017-01-01 07:38:12.253,2017-01-01 09:09:51.681,2017-01-01 09:18:02.746,2017-01-01 09:16:23.235,491.064347,Stripmap S1
10,2017-01-01 09:16:23.235,2017-01-01 09:46:41.020,2017-01-01 09:49:12.943,2017-01-01 10:54:34.216,151.922865,Stripmap S1
11,2017-01-01 09:16:23.235,2017-01-01 09:50:14.404,2017-01-01 09:53:27.306,2017-01-01 10:54:34.216,192.902257,Stripmap S1
12,2017-01-01 09:16:23.235,2017-01-01 09:55:51.239,2017-01-01 09:56:23.239,2017-01-01 10:54:34.216,32.000000,Stripmap S1


ID of selected acquisition: 124

In [38]:
acq_idx = 2

In [39]:
data_take = pd.Interval(simulation.loc[acq_idx, 'Start'], simulation.loc[acq_idx, 'End'], closed='both')
data_take

Interval('2017-01-01 06:52:49.335000', '2017-01-01 06:55:03.761000', closed='both')

In [40]:
dtid = simulation.loc[acq_idx, 'DTID']
dtid

11360

#### Swath

In [41]:
# get swath id from SensorMode name
swath_id = simulation.loc[acq_idx]['SensorMode'][-2:]
swath_id

'S1'

In [42]:
# crosscheck with orbit number
AbsoluteOrbit(simulation.loc[acq_idx]['Orbit'], tm.MissionPhase.INT).swathid()

1

#### Global coverage id, major cycle id, repeat cycle id, drift flag, orbit and track

In [43]:
gcid = simulation.loc[acq_idx]['gcid']
mcid = simulation.loc[acq_idx]['mcid']
rcid = simulation.loc[acq_idx]['rcid']
track = simulation.loc[acq_idx]['track']

In [44]:
dt_polygon = simulation.loc[acq_idx, ['NW_Lat', 'NW_Lon', 'NE_Lat', 'NE_Lon', 'SE_Lat', 'SE_Lon', 'SW_Lat', 'SW_Lon', 'NW_Lat', 'NW_Lon']].to_numpy()

footprint_polygon = ' '.join(dt_polygon.astype(str))

#### Acquisition

Acquisition date is set to 30 minutes after data take stop

In [45]:
acquisition_date = data_take.right + pd.to_timedelta(30, 'm')
acquisition_station = 'SV'  # Svalbard
acquisition_date # start

Timestamp('2017-01-01 07:25:03.761000')

#### Affected Orbits

In [46]:
orbits = get_orbits(simulation, acq_idx)
orbits

[Interval('2017-01-01 06:00:01.272000', '2017-01-01 07:38:12.253000', closed='both')]

#### Level 0 slices

In [47]:
slices = get_slices(orbits, data_take, SLICE_MIN_DURATION)
slices

,n,start,stop,duration
0,34,2017-01-01 06:52:49.335000000,2017-01-01 06:53:58.809463160,69.474463
1,35,2017-01-01 06:53:46.809463160,2017-01-01 06:55:03.761000000,76.951536


In case of platform ancillary data, validity starts 16 seconds before data take start:

In [48]:
plt_data_validity = pd.Interval(
    data_take.left - pd.to_timedelta(const.PLATFORM_ANCILLARY_INITIAL_MARGIN, 's'),
    data_take.right + pd.to_timedelta(const.PLATFORM_ANCILLARY_FINAL_MARGIN, 's'),
    closed='both'
)
plt_slices = get_slices(orbits, plt_data_validity, SLICE_MIN_DURATION)
plt_slices

,n,start,stop,duration
0,34,2017-01-01 06:52:33.335000000,2017-01-01 06:53:58.809463160,85.474463
1,35,2017-01-01 06:53:46.809463160,2017-01-01 06:55:03.761000000,76.951536


#### Level 1 frames

In [49]:
frames = get_frames(orbits, data_take, FRAME_MIN_DURATION)
frames

,n,start,stop,duration
0,167,2017-01-01 06:52:49.335000000,2017-01-01 06:52:56.799978516,7.464978
1,168,2017-01-01 06:52:54.799978516,2017-01-01 06:53:15.803140064,21.003161
2,169,2017-01-01 06:53:13.803140064,2017-01-01 06:53:34.806301612,21.003161
3,170,2017-01-01 06:53:32.806301612,2017-01-01 06:53:53.809463160,21.003161
4,171,2017-01-01 06:53:51.809463160,2017-01-01 06:54:12.812624708,21.003161
5,172,2017-01-01 06:54:10.812624708,2017-01-01 06:54:31.815786256,21.003161
6,173,2017-01-01 06:54:29.815786256,2017-01-01 06:54:50.818947804,21.003161
7,174,2017-01-01 06:54:48.818947804,2017-01-01 06:55:03.761000000,14.942052


#### Additional data takes for stack

Select additional data takes having the same global coverage id, major cycle id, swath (sensor mode) and track:

In [50]:
def overlap(data_take_start, data_take_stop, orbit_start, orbit_stop, frame_numbers):
    data_take = pd.Interval(data_take_start, data_take_stop, closed='both')
    frame_grid = FrameGrid(orbit_start, orbit_stop)
    for frame_number in frame_numbers:
        overlap = frame_grid.interval(frame_number).overlaps(data_take)
        if overlap is False:
            return False
    return True

sensor_mode = simulation.loc[acq_idx]['SensorMode']
track = simulation.loc[acq_idx]['track']
result = simulation.query("gcid == @gcid and mcid == @mcid and SensorMode == @sensor_mode and track == @track")
# Select only data takes having overlaps with frame numbers selected above
idx = result.apply(
    lambda r: overlap(r['Start'], r['End'], r['Anx'], r['NextAnx'], frames['n']),
    axis=1
)
result[idx]

,Start,End,Duration,Satellite,SensorMode,Width,Length,SwathArea,Pass,NW_Lat,...,Orbit,Anx,AnxLon,TAnx,DTID,gcid,mcid,rcid,track,NextAnx
2,2017-01-01 06:52:49.335,2017-01-01 06:55:03.761,134.425245,BIOMASS,Stripmap S1,54.466270,920.063964,50254.777249,DESCENDING,-14.294510,...,1,2017-01-01 06:00:01.272,-1.141720e-07,3168.063289,11360,1,1,1,1,2017-01-01 07:38:12.253
217,2017-01-04 06:52:52.438,2017-01-04 06:55:06.805,134.366732,BIOMASS,Stripmap S1,54.466132,919.664998,50246.328664,DESCENDING,-14.289509,...,45,2017-01-04 06:00:04.459,-1.328532e-02,3167.979528,371807,1,1,2,1,2017-01-04 07:38:15.440
431,2017-01-07 06:52:55.587,2017-01-07 06:55:09.850,134.263520,BIOMASS,Stripmap S1,54.466155,918.959438,50210.451058,DESCENDING,-14.287216,...,89,2017-01-07 06:00:07.645,-2.656646e-02,3167.941390,732255,1,1,3,1,2017-01-07 07:38:18.626


In [51]:
idxs = list(result[idx].index.to_numpy())
idxs

[2, 217, 431]

#### Raw Data generation

Scenario generation

In [52]:
config = {
    'mission': 'biomass',
    'scenarios': [raw_generation_scenario('INT_TDS', simulation, idx, 'MM') for idx in idxs]
}
with open('int-tds-raw-gen-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)

#### Level 0

Scenario generation

In [53]:
config = {
    'mission': 'biomass',
    'scenarios': []
}
for idx in idxs:
    config['scenarios'].extend(l0pfs1_processing_scenarios('INT_TDS', simulation, idx, 'MM', 'INTERFEROMETRIC'))
    config['scenarios'].append(l0pfs2_processing_scenario('INT_TDS', simulation, idx, 'MM', 'INTERFEROMETRIC'))
    config['scenarios'].append(l0pfs3_processing_scenario('INT_TDS', simulation, idx, 'MM', 'INTERFEROMETRIC'))
    config['scenarios'].append(l0pfs4_processing_scenario('INT_TDS', simulation, idx, 'MM', 'INTERFEROMETRIC'))
    config['scenarios'].append(oapf_processing_scenario('INT_TDS', simulation, idx, 'MM', 'INTERFEROMETRIC'))
with open('int-tds-l0-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)

#### Level 1a/b

In [54]:
config = {
    'mission': 'biomass',
    'scenarios': []
}
for idx in idxs:
    config['scenarios'].extend(l1_processing_scenarios('INT_TDS', simulation, idx, 'INTERFEROMETRIC'))
with open('int-tds-l1-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)

#### Level 1c 

In [55]:
config = {
    'mission': 'biomass',
    'scenarios': []
}
for idx in idxs:
    config['scenarios'].extend(l1c_processing_scenarios('INT_TDS', simulation, idx, 'INTERFEROMETRIC'))
with open('int-tds-l1c-l2a-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)

### RO scenario

#### Data Take selection

In [56]:
columns = ['Anx', 'Start', 'End', 'NextAnx', 'Duration', 'gcid', 'mcid', 'rcid', 'Orbit', 'track']
simulation.query("SensorMode == 'Receive Only' and 100 < Duration < 150")[columns].head()

,Anx,Start,End,NextAnx,Duration,gcid,mcid,rcid,Orbit,track
7095,2017-05-25 16:10:46.176,2017-05-25 16:43:47.078,2017-05-25 16:46:15.045,2017-05-25 17:48:57.158,147.967056,1,5,1,2119,7
7307,2017-05-28 16:10:49.360,2017-05-28 16:43:50.201,2017-05-28 16:46:18.455,2017-05-28 17:49:00.345,148.254601,1,5,2,2163,7
7519,2017-05-31 16:10:52.549,2017-05-31 16:43:53.319,2017-05-31 16:46:21.878,2017-05-31 17:49:03.530,148.558840,1,5,3,2207,7


ID of selected acquisition: 7095

In [57]:
acq_idx = 7095

In [58]:
data_take = pd.Interval(simulation.loc[acq_idx, 'Start'], simulation.loc[acq_idx, 'End'], closed='both')
data_take

Interval('2017-05-25 16:43:47.078000', '2017-05-25 16:46:15.045000', closed='both')

In [59]:
dtid = simulation.loc[acq_idx, 'DTID']
dtid

17360828

#### Swath

In [60]:
swath_id = 'RO'

#### Global coverage id, major cycle id, repeat cycle id, drift flag, orbit and track

In [61]:
gcid = simulation.loc[acq_idx]['gcid']
mcid = simulation.loc[acq_idx]['mcid']
rcid = simulation.loc[acq_idx]['rcid']
track = simulation.loc[acq_idx]['track']

In [62]:
dt_polygon = simulation.loc[acq_idx, ['NW_Lat', 'NW_Lon', 'NE_Lat', 'NE_Lon', 'SE_Lat', 'SE_Lon', 'SW_Lat', 'SW_Lon', 'NW_Lat', 'NW_Lon']].to_numpy()

footprint_polygon = ' '.join(dt_polygon.astype(str))

#### Acquisition

Acquisition date is set to 30 minutes after data take stop

In [63]:
acquisition_date = data_take.right + pd.to_timedelta(30, 'm')
acquisition_station = 'SV'  # Svalbard
acquisition_date # start

Timestamp('2017-05-25 17:16:15.045000')

#### Affected Orbits

In [64]:
orbits = get_orbits(simulation, acq_idx)
orbits

[Interval('2017-05-25 16:10:46.176000', '2017-05-25 17:48:57.158000', closed='both')]

#### Level 0 slices

In [65]:
slices = get_slices(orbits, data_take, SLICE_MIN_DURATION)
slices

,n,start,stop,duration
0,22,2017-05-25 16:43:47.078000000,2017-05-25 16:45:43.523770280,116.445770
1,23,2017-05-25 16:45:31.523770280,2017-05-25 16:46:15.045000000,43.521229


In case of platform ancillary data, validity starts 16 seconds before data take start:

In [66]:
plt_data_validity = pd.Interval(
    data_take.left - pd.to_timedelta(const.PLATFORM_ANCILLARY_INITIAL_MARGIN, 's'),
    data_take.right + pd.to_timedelta(const.PLATFORM_ANCILLARY_FINAL_MARGIN, 's'),
    closed='both'
)
plt_slices = get_slices(orbits, plt_data_validity, SLICE_MIN_DURATION)
plt_slices

,n,start,stop,duration
0,21,2017-05-25 16:43:31.078000000,2017-05-25 16:44:08.507962540,37.429962
1,22,2017-05-25 16:43:56.507962540,2017-05-25 16:45:43.523770280,107.015807
2,23,2017-05-25 16:45:31.523770280,2017-05-25 16:46:15.045000000,43.521229


#### Level 1 frames

In [67]:
frames = get_frames(orbits, data_take, FRAME_MIN_DURATION)
frames

,n,start,stop,duration
0,105,2017-05-25 16:43:47.078000000,2017-05-25 16:44:03.507962540,16.429962
1,106,2017-05-25 16:44:01.507962540,2017-05-25 16:44:22.511124088,21.003161
2,107,2017-05-25 16:44:20.511124088,2017-05-25 16:44:41.514285636,21.003161
3,108,2017-05-25 16:44:39.514285636,2017-05-25 16:45:00.517447184,21.003161
4,109,2017-05-25 16:44:58.517447184,2017-05-25 16:45:19.520608732,21.003161
5,110,2017-05-25 16:45:17.520608732,2017-05-25 16:45:38.523770280,21.003161
6,111,2017-05-25 16:45:36.523770280,2017-05-25 16:45:57.526931828,21.003161
7,112,2017-05-25 16:45:55.526931828,2017-05-25 16:46:15.045000000,19.518068


#### Raw Data generation

Scenario generation

In [68]:
with open('int-tds-raw-gen-scenarios.json') as fh:
    config = json.load(fh)

config['scenarios'].append(raw_generation_scenario('INT_TDS', simulation, acq_idx, 'RO'))

with open('int-tds-raw-gen-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)

#### Level 0

Scenario generation

In [69]:
with open('int-tds-l0-scenarios.json') as fh:
    config = json.load(fh)

config['scenarios'].extend(l0pfs1_processing_scenarios('INT_TDS', simulation, acq_idx, 'RO', 'INTERFEROMETRIC'))
config['scenarios'].append(l0pfs2_processing_scenario('INT_TDS', simulation, acq_idx, 'RO', 'INTERFEROMETRIC'))
config['scenarios'].append(l0pfs4_processing_scenario('INT_TDS', simulation, acq_idx, 'RO', 'INTERFEROMETRIC'))
config['scenarios'].append(oapf_processing_scenario('INT_TDS', simulation, acq_idx, 'RO', 'INTERFEROMETRIC'))

with open('int-tds-l0-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)

## AUX Data generation

Scenario generation

In [70]:
baseline = 1

config = {
    'mission': 'biomass',
    'scenarios': [
        {
            'name': file_type,
            'file_name': 'N/A',
            'processor_name': 'procsim',
            'processor_version': procsim_version,
            'task_name': 'N/A',
            'task_version': 'N/A',
            'log_level': 'debug',
            'begin_position': '2000-01-01T00:00:00.000Z',
            'end_position': '2050-01-01T00:00:00.000Z',
            'baseline': baseline,
            'outputs': [
                {'type': file_type}
            ]
        }
        for file_type in ('AUX_INS___', 'AUX_PP1___', 'AUX_PPS___', 'AUX_PP2_2A')  # add AUX_PP0___ when supported
    ]
}

# Processing Parameters
config['scenarios'].append(
    {
        'name': 'all',
        'file_name': 'N/A',
        'processor_name': 'procsim',
        'processor_version': procsim_version,
        'task_name': 'N/A',
        'task_version': 'N/A',
        'log_level': 'debug',
        'begin_position': '2000-01-01T00:00:00.000Z',
        'end_position': '2050-01-01T00:00:00.000Z',
        'baseline': baseline,
        'outputs': [
            {'type': file_type}
            for file_type in ('AUX_INS___', 'AUX_PP1___', 'AUX_PPS___', 'AUX_PP2_2A')  # add AUX_PP0___ when supported
        ]
    }
)

In [71]:
with open('aux-gen-scenarios.json', 'w') as fh:
    json.dump(config, fh, indent=2)